In [226]:
#%pip install spacy -qq

#POC model that is able to extract locations (to and from), dates and prices ranges from conversational text.

import nltk, nltk.tag, nltk.chunk
import spacy
import pprint as pprint
from gensim.summarization import summarize
from collections import Counter

#import en_core_web_sm
#nlp = en_core_web_sm.load()

from bs4 import BeautifulSoup
import requests
import re
from spacy import displacy


In [227]:
#If I want to use a prepackaged NLP

#ny_bb = 'Tlg bawa kami graduan dan family pulang.. They came for our convo but then cancelled unfortunately our flight has been cancelled too because egypt had suspended all international flight in and out. Most of airlines may not be operating till june'
#article = nlp(ny_bb)
#labels = [x.label_ for x in article.ents]
#Counter(labels)
#sentences = [x for x in article.sents]
#displacy.render(nlp(str(sentences[:])), jupyter=True, style='ent')
#print(sentences[40])

# Build My own NLP

## Create Training Data

In [228]:
nltk.download('nps_chat')

from nltk.corpus import nps_chat
chatroom = nps_chat.posts('10-19-20s_706posts.xml')

[nltk_data] Downloading package nps_chat to /home/dan/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


In [229]:
#for t in chatroom:
#    print(t)

In [230]:
Sample_Text_1 = [['Hi there'],
               ['How can I help you?'],
               ['Are there any flights from Houston to Boston'],
               ['Yes, for which dates would you like?'],
               ['June 3rd to June 18th'],
               ['How much would like to spend?'],
               ['Under 500'],
               ['Great, Here are some options:']]


Sample_Text_2 = [['Good Morning'],
               ['How can i help you?'],
               ['I would like to see if I can travel from Seattle to Chicago tomorrow'],
               ['When would you like to come back'],
               ['In two weeks, but I am flexible on the day'],
               ['What is your budget'],
               ['Less than fifteen hundred dollars'],
               ['Great, Here are some options:']]
               
Sample_Text_3 = [['Yo'],
               ['How can I help you?'],
               ['How much is it to go from San Diego to Los Angeles. I have to get there for a wedding.'],
               ['When would you like to travel'],
               ['March 1st to March 2nd'],
               ['What is your budget'],
               ['The cheapest you can find'],
               ['Great, Here are some options:']]

Sample_Text_4 = [['Guten Tag'],
               ['How can I help you?'],
               ['I would like to fly to San Francisco as soon as possible'],
               ['What is your budget'],
               ['Price is no object, and I would like first class if possible'],
               ['Great, Here are some options:']]

Sample_Text_5 = [['Hello'],
               ['How can I help you?'],
               ['I would like to fly to Portland as soon as possible'],
               ['Where are you located'],
               ['Washington'],
               ['What is your budget'],
               ['Between $400 and $500'],
               ['Great, Here are some options:']]


Complete_Sample = Sample_Text_1 + Sample_Text_2 + Sample_Text_3 + Sample_Text_4 + Sample_Text_5

#Complete_Sample = Sample_Text_1

In [231]:
def is_number(x):
    if type(x) == str:
        x = x.replace(',', '')
    try:
        float(x)
    except:
        return False
    return True

def text2int (textnum, numwords={}):
    units = [
        'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
        'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen',
        'sixteen', 'seventeen', 'eighteen', 'nineteen',
    ]
    tens = ['', '', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety']
    scales = ['hundred', 'thousand', 'million', 'billion', 'trillion']
    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
    ordinal_endings = [('ieth', 'y'), ('th', '')]

    if not numwords:
        numwords['and'] = (1, 0)
        for idx, word in enumerate(units): numwords[word] = (1, idx)
        for idx, word in enumerate(tens): numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)

    textnum = textnum.replace('-', ' ')

    current = result = 0
    curstring = ''
    onnumber = False
    lastunit = False
    lastscale = False

    def is_numword(x):
        if is_number(x):
            return True
        if word in numwords:
            return True
        return False

    def from_numword(x):
        if is_number(x):
            scale = 0
            increment = int(x.replace(',', ''))
            return scale, increment
        return numwords[x]

    for word in textnum.split():
        if word in ordinal_words:
            scale, increment = (1, ordinal_words[word])
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0
            onnumber = True
            lastunit = False
            lastscale = False
        else:
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)

            if (not is_numword(word)) or (word == 'and' and not lastscale):
                if onnumber:
                    # Flush the current number we are building
                    curstring += repr(result + current) + " "
                curstring += word + " "
                result = current = 0
                onnumber = False
                lastunit = False
                lastscale = False
            else:
                scale, increment = from_numword(word)
                onnumber = True

                if lastunit and (word not in scales):                                                                                                                                                                                                                                         
                    # Assume this is part of a string of individual numbers to                                                                                                                                                                                                                
                    # be flushed, such as a zipcode "one two three four five"                                                                                                                                                                                                                 
                    curstring += repr(result + current)                                                                                                                                                                                                                                       
                    result = current = 0                                                                                                                                                                                                                                                      

                if scale > 1:                                                                                                                                                                                                                                                                 
                    current = max(1, current)                                                                                                                                                                                                                                                 

                current = current * scale + increment                                                                                                                                                                                                                                         
                if scale > 100:                                                                                                                                                                                                                                                               
                    result += current                                                                                                                                                                                                                                                         
                    current = 0                                                                                                                                                                                                                                                               

                lastscale = False                                                                                                                                                                                                              
                lastunit = False                                                                                                                                                
                if word in scales:                                                                                                                                                                                                             
                    lastscale = True                                                                                                                                                                                                         
                elif word in units:                                                                                                                                                                                                             
                    lastunit = True

    if onnumber:
        curstring += repr(result + current)

    return curstring
 
Complete_Sample_2 = [[text2int(sent[0])] for sent in Complete_Sample]   

cnt = 0
for cnt in np.arange(0,len(Complete_Sample_2)):
    Complete_Sample_2[cnt][0] = Complete_Sample_2[cnt][0].replace('as soon as possible','today')
    Complete_Sample_2[cnt][0] = Complete_Sample_2[cnt][0].replace('asap','today')
    Complete_Sample_2[cnt][0] = Complete_Sample_2[cnt][0].replace('ASAP','today')
    
    
Complete_Sample_2

[['Hi there '],
 ['How can I help you? '],
 ['Are there any flights from Houston to Boston '],
 ['Yes, for which dates would you like? '],
 ['June 3rd to June 18'],
 ['How much would like to spend? '],
 ['Under 500'],
 ['Great, Here are some options: '],
 ['Good Morning '],
 ['How can i help you? '],
 ['I would like to see if I can travel from Seattle to Chicago tomorrow '],
 ['When would you like to come back '],
 ['In 2 weeks, but I am flexible on the day '],
 ['What is your budget '],
 ['Less than 1500 dollars '],
 ['Great, Here are some options: '],
 ['Yo '],
 ['How can I help you? '],
 ['How much is it to go from San Diego to Los Angeles. I have to get there for a wedding. '],
 ['When would you like to travel '],
 ['March 1st to March 2nd '],
 ['What is your budget '],
 ['The cheapest you can find '],
 ['Great, Here are some options: '],
 ['Guten Tag '],
 ['How can I help you? '],
 ['I would like to fly to San Francisco today '],
 ['What is your budget '],
 ['Price is no object, a

## Tokenizer

In [232]:
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize

token_sent = []

for response in Complete_Sample_2:
    token_sent.append(sent_tokenize(response[0]))

print(token_sent[0])

['Hi there']


In [233]:
token_word = []
from nltk.tokenize import word_tokenize
for word in token_sent:
    token_word.append(word_tokenize(str(word[0])))

print(token_word)

[['Hi', 'there'], ['How', 'can', 'I', 'help', 'you', '?'], ['Are', 'there', 'any', 'flights', 'from', 'Houston', 'to', 'Boston'], ['Yes', ',', 'for', 'which', 'dates', 'would', 'you', 'like', '?'], ['June', '3rd', 'to', 'June', '18'], ['How', 'much', 'would', 'like', 'to', 'spend', '?'], ['Under', '500'], ['Great', ',', 'Here', 'are', 'some', 'options', ':'], ['Good', 'Morning'], ['How', 'can', 'i', 'help', 'you', '?'], ['I', 'would', 'like', 'to', 'see', 'if', 'I', 'can', 'travel', 'from', 'Seattle', 'to', 'Chicago', 'tomorrow'], ['When', 'would', 'you', 'like', 'to', 'come', 'back'], ['In', '2', 'weeks', ',', 'but', 'I', 'am', 'flexible', 'on', 'the', 'day'], ['What', 'is', 'your', 'budget'], ['Less', 'than', '1500', 'dollars'], ['Great', ',', 'Here', 'are', 'some', 'options', ':'], ['Yo'], ['How', 'can', 'I', 'help', 'you', '?'], ['How', 'much', 'is', 'it', 'to', 'go', 'from', 'San', 'Diego', 'to', 'Los', 'Angeles', '.'], ['When', 'would', 'you', 'like', 'to', 'travel'], ['March', '

In [234]:
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

stop_words

import string

no_punct = [] 
            
for t in token_word:
    for ts in t:
        if ts not in string.punctuation:
            no_punct.append(ts)

no_punct
data_lower = [w.lower() for w in no_punct]

#data_lower
filtered_sent=[]

for w in data_lower:
    if w not in stop_words:
        filtered_sent.append(w)

print(filtered_sent)

['hi', 'help', 'flights', 'houston', 'boston', 'yes', 'dates', 'would', 'like', 'june', '3rd', 'june', '18', 'much', 'would', 'like', 'spend', '500', 'great', 'options', 'good', 'morning', 'help', 'would', 'like', 'see', 'travel', 'seattle', 'chicago', 'tomorrow', 'would', 'like', 'come', 'back', '2', 'weeks', 'flexible', 'day', 'budget', 'less', '1500', 'dollars', 'great', 'options', 'yo', 'help', 'much', 'go', 'san', 'diego', 'los', 'angeles', 'would', 'like', 'travel', 'march', '1st', 'march', '2nd', 'budget', 'cheapest', 'find', 'great', 'options', 'guten', 'tag', 'help', 'would', 'like', 'fly', 'san', 'francisco', 'today', 'budget', 'price', 'object', 'would', 'like', '1', 'class', 'possible', 'great', 'options', 'hello', 'help', 'would', 'like', 'fly', 'portland', 'today', 'located', 'washington', 'budget', '400', '500', 'great', 'options']


In [235]:
#Stemmer and Lemmatizer

#from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer

#ps= PorterStemmer()
#ls = LancasterStemmer()

#psstemmed_words=[]
#lsstemmed_words = []

#for w in filtered_sent:    
    #psstemmed_words.append(ps.stem(w))
    #lsstemmed_words.append(ls.stem(w))
    #print("Words "+w+ "    PS :"+ps.stem(w) + "     LS : "+ls.stem(w))

    
#from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('wordnet')

#lem = WordNetLemmatizer()
#for word in filtered_sent:
#    print ("{0:20}{1:20}".format(word,lem.lemmatize(word, pos ="v")))

#today = date.today()
#d1 = today.strftime("%B %d")

#print(str(d1))

## Data Cleaning Massaging

In [236]:
#Data Cleaning / Massaging

# Places -> Fine

# Time and Dates
import datetime
from datetime import date

today = date.today()
tomorrow = date.today() + datetime.timedelta(days=1)

filtered_sent_2 = filtered_sent.copy()
cnt = 0 

for word in filtered_sent:
    if word[-2:] == 'th' or word[-2:] == 'st' or word[-2:] == 'rd' or word[-2:] == 'nd':
        try:
            int(word[0])
            print(word[:-2])
            filtered_sent_2[cnt] = word[:-2]

        except:
            cnt2=1
            
    if word == 'tomorrow':        
        filtered_sent_2[cnt] =  tomorrow.strftime("%d-%m-%Y")
        #print(d2)
        
    if word == 'today' or word == 'now':        
        filtered_sent_2[cnt] = today.strftime("%d-%m-%Y")       
        #print(d1)    
        
    cnt = cnt +1

#filtered_sent_2

3
1
2


['hi',
 'help',
 'flights',
 'houston',
 'boston',
 'yes',
 'dates',
 'would',
 'like',
 'june',
 '3',
 'june',
 '18',
 'much',
 'would',
 'like',
 'spend',
 '500',
 'great',
 'options',
 'good',
 'morning',
 'help',
 'would',
 'like',
 'see',
 'travel',
 'seattle',
 'chicago',
 '25-03-2020',
 'would',
 'like',
 'come',
 'back',
 '2',
 'weeks',
 'flexible',
 'day',
 'budget',
 'less',
 '1500',
 'dollars',
 'great',
 'options',
 'yo',
 'help',
 'much',
 'go',
 'san',
 'diego',
 'los',
 'angeles',
 'would',
 'like',
 'travel',
 'march',
 '1',
 'march',
 '2',
 'budget',
 'cheapest',
 'find',
 'great',
 'options',
 'guten',
 'tag',
 'help',
 'would',
 'like',
 'fly',
 'san',
 'francisco',
 '24-03-2020',
 'budget',
 'price',
 'object',
 'would',
 'like',
 '1',
 'class',
 'possible',
 'great',
 'options',
 'hello',
 'help',
 'would',
 'like',
 'fly',
 'portland',
 '24-03-2020',
 'located',
 'washington',
 'budget',
 '400',
 '500',
 'great',
 'options']

## Tagger

In [237]:
#dl_pos =  nltk.pos_tag(data_lower)
#dl_pos = default_tagger.tag(data_lower)

In [238]:
#nltk.download('averaged_perceptron_tagger')
#nltk.pos_tag(data_lower)
from nltk.tag import UnigramTagger
from nltk.corpus import brown

brown_tagged_sents = brown.tagged_sents(categories='news')

size = int(len(brown_tagged_sents) * 0.9)

train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)
#t2.tag(filtered_sent_2)


## Parsers

https://www.nltk.org/api/nltk.parse.html

Run specific parsers according to each type of information we want to extract?

## Locations

In [239]:
### Locations
#https://www.geeksforgeeks.org/nlp-location-tags-extraction/
import numpy as np
from nltk.chunk import ChunkParserI 
from nltk.chunk.util import conlltags2tree 
from nltk.corpus import gazetteers 
  
#sent_pos = nltk.pos_tag(data_lower)    
words_tagged = t2.tag(filtered_sent_2)
place_lower = [w.lower() for w in gazetteers.words()]

loc_tag = words_tagged

cnt=0
for cnt in np.arange(1,len(words_tagged)-1):
    if words_tagged[cnt][0] in place_lower:
        if words_tagged[cnt][1] == 'NN':
            print(words_tagged[cnt][0])
            loc_tag[cnt] = (words_tagged[cnt][0],'LOCATION') 
            
    link_place = words_tagged[cnt][0] + ' ' + words_tagged[cnt+1][0]        
    if link_place in place_lower:      
        if words_tagged[cnt][1] in ['JJ','NN'] and words_tagged[cnt+1][1] == 'NN':
            print(link_place)
            loc_tag[cnt] = (link_place,'LOCATION') 
            
    cnt=cnt+1

#print(loc_tag)
#'/' in '25/03/2020'

houston
boston
seattle
chicago
san diego
los angeles
san francisco
portland
washington


## Time and Dates and Money


In [240]:
#https://stackoverflow.com/questions/31088509/identifying-dates-in-strings-using-nltk
#from nltk.tag import StanfordNERTagger
#stanfordClassifier = '/path/to/classifier/classifiers/english.muc.7class.distsim.crf.ser.gz'
#stanfordNerPath = '/path/to/jar/stanford-ner-2017-06-09/stanford-ner.jar'
#st = StanfordNERTagger(stanfordClassifier, stanfordNerPath, encoding='utf8')

#result = st.tag(data_lower)
#print(result)

time_tag = loc_tag.copy()

month_lower = ['january','february','march','april','may','june',
              'july','august','september','october','november','december',
              'jan','feb','mar','apr','may','jun',
              'jul','aug','sep','oct','nov','dec','week','weeks','month']

cnt=0
for cnt in np.arange(1,len(loc_tag)-1):
    
    if '-' in loc_tag[cnt][0]:
                time_tag[cnt] = (loc_tag[cnt][0],'DATETIME') 
    
    if loc_tag[cnt][0] in month_lower:
        if loc_tag[cnt][1] == 'NN' or loc_tag[cnt][1] == 'NNS':
                   
            date_p2 = loc_tag[cnt-1][0]        
            try:
                int(date_p2)
                time_tag[cnt] = (loc_tag[cnt][0] + ' ' + date_p2 ,'DATETIME') 
                #time_tag[cnt-1] = ('','NN') 
            except:
                cnt2=1    
                
            date_p1 = loc_tag[cnt+1][0]        
            try:
                int(date_p1)
                time_tag[cnt] = (loc_tag[cnt][0] + ' ' + date_p1,'DATETIME') 
                #time_tag[cnt+1] = ('','NN') 
            except:
                cnt2=1
    cnt=cnt+1

#print(loc_tag)

grammar = 'NumPhrase: {<CD><CD|NNS|JJ>}'
t_parser = nltk.RegexpParser(grammar)

final_tree = t_parser.parse(time_tag)

In [241]:
print(type(final_tag))
print(final_tree)

final_tags  = tree2conlltags(final_tree)
print(final_tags)

<class 'nltk.tree.Tree'>
(S
  hi/NN
  help/VB
  flights/NNS
  houston/LOCATION
  boston/LOCATION
  yes/NN
  dates/NNS
  would/MD
  like/VB
  june 3/DATETIME
  3/CD
  june 18/DATETIME
  18/CD
  much/AP
  would/MD
  like/VB
  spend/VB
  (NumPhrase 500/CD great/JJ)
  options/NN
  good/NN
  morning/NN
  help/VB
  would/MD
  like/VB
  see/VB
  travel/NN
  seattle/LOCATION
  chicago/LOCATION
  25-03-2020/DATETIME
  would/MD
  like/VB
  come/VB
  back/RB
  2/CD
  weeks 2/DATETIME
  flexible/JJ
  day/NN
  budget/NN
  less/QL
  (NumPhrase 1500/CD dollars/NNS)
  great/JJ
  options/NN
  yo/NN
  help/VB
  much/AP
  go/VB
  san diego/LOCATION
  diego/NN
  los angeles/LOCATION
  angeles/NN
  would/MD
  like/VB
  travel/NN
  march 1/DATETIME
  1/CD
  march 2/DATETIME
  2/CD
  budget/NN
  cheapest/NN
  find/VB
  great/JJ
  options/NN
  guten/NN
  tag/NN
  help/VB
  would/MD
  like/VB
  fly/NN
  san francisco/LOCATION
  francisco/NN
  24-03-2020/DATETIME
  budget/NN
  price/NN
  object/NN
  would/MD
  

In [242]:
#Find all important tags
from nltk import conlltags2tree, tree2conlltags

Locations = [[tag[0]] for tag in final_tags if tag[1] == 'LOCATION']  
Locations

Dates = [[tag[0]] for tag in final_tags if tag[1] == 'DATETIME']  
Dates

Money = [[tag[0]] for tag in final_tags if tag[2] == 'B-NumPhrase']  
Money

print(Locations,Dates,Money)


[['houston'], ['boston'], ['seattle'], ['chicago'], ['san diego'], ['los angeles'], ['san francisco'], ['portland'], ['washington']] [['june 3'], ['june 18'], ['25-03-2020'], ['weeks 2'], ['march 1'], ['march 2'], ['24-03-2020'], ['24-03-2020']] [['500'], ['1500'], ['400']]


## NER
Chunking?

In [243]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk, pos_tag
chunked = ne_chunk(time_tag)

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/dan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/dan/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [244]:
#print(chunked)

## Text Categorizer

## Custom Components

## Logic Engine to parse NE

In [245]:
# If no second loaction, ask for start
# If no second date, assume one way
# If no money, assume cheapest
# If no dates, ask for dates, if not look at today/oneway